In [1]:
import dxdata
import dxpy
import hail as hl

import pyspark
import tomli
import subprocess
from matrixtables import *
from utils import get_stats
from datetime import datetime
from pprint import pprint

Path("../tmp").resolve().mkdir(parents=True, exist_ok=True)

In [2]:
# Parameters
with open("../config.toml", "rb") as f:
    conf = tomli.load(f)

IMPORT = conf["IMPORT"]
NAME = conf["NAME"]
VCF_VERSION = IMPORT["VCF_VERSION"]
REFERENCE_GENOME = conf["REFERENCE_GENOME"]
DATABASE = IMPORT["DATABASE"]

LOG_FILE = (
    Path(IMPORT["LOG_DIR"], f"{NAME}_{datetime.now().strftime('%H%M')}.log")
    .resolve()
    .__str__()
)

GENE_FILE = Path(IMPORT["GENE_FILE"]).resolve().__str__()

with open(GENE_FILE, "r") as file:
    genes = file.read().splitlines()
    
if NAME == "NONE":
    NAME = genes[0]

In [3]:
# Spark and Hail

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]
hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-12-76.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/hail_logs/NONE_1433.log


In [4]:
STAGE = "QC1"
READ_PATH = "dnax://" + mt_database + f"/{NAME}.{STAGE}.mt"

mt = hl.read_matrix_table(READ_PATH)

v, s = mt.count()
pprint(f"{v} variants and {s} samples after reading matrixtable")

'223 variants and 200643 samples after reading matrixtable'


In [97]:
ht = hl.import_table("file:" + "/mnt/project/Data/annotations/MC4R_annotations.tsv", impute=True, quote = '"')
ht = ht.annotate(Variants = ht.Variants.strip())
ht = ht.key_by("Variants")

2021-12-06 15:15:07 Hail: INFO: Reading table to impute column types
2021-12-06 15:15:08 Hail: INFO: Finished type imputation
  Loading field 'Variants' as type str (imputed)
  Loading field 'Category' as type int32 (imputed)


In [96]:
ht.Variants.collect()

["5'UTR 32bp vor A+G:A>C",
 'A135P',
 'A175T',
 'C271R',
 'D90N',
 'E61*',
 'G181D',
 'I170V',
 'I198I',
 'I251L',
 'N274S',
 'P272L',
 'P275S',
 'P48S',
 'S127L',
 'S77L',
 'S77L',
 'S85G+Y268H',
 'T112K',
 'T112M',
 'T11l',
 'T122T',
 'T178M',
 'V103I',
 'V103l+S127L',
 'V166I',
 'V166L',
 'Y35X+D37V',
 'Y80C',
 'c.1000+16 G>A',
 'c.811insT, p.C271fsX285']

In [82]:
mt = mt.annotate_rows(labels = ht[mt.protCons].Category)

In [83]:
mt.aggregate_rows(hl.agg.sum(hl.is_defined(mt.labels)))

2021-12-06 15:08:08 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:08:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:08:09 Hail: INFO: Ordering unsorted dataset with network shuffle


14

In [85]:
found = mt.filter_rows(hl.is_defined(mt.labels)).rows()
found.show(-1)

2021-12-06 15:09:27 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:27 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:09:32 Hail: INFO: Ordering unsorted dataset with network shuffle


+----------------+------------+-------------------------------------------+
| locus          | alleles    | rsid                                      |
+----------------+------------+-------------------------------------------+
| locus<GRCh38>  | array<str> | str                                       |
+----------------+------------+-------------------------------------------+
| chr18:60371527 | ["G","A"]  | "chr18_60371527_G_A"                      |
| chr18:60371529 | ["T","C"]  | "chr18_60371529_T_C"                      |
| chr18:60371539 | ["A","G"]  | "chr18_60371539_A_G"                      |
| chr18:60371599 | ["T","G"]  | "chr18_60371599_T_G;chr18_60371598_ATC_A" |
| chr18:60371756 | ["G","A"]  | "chr18_60371756_G_A"                      |
| chr18:60371808 | ["C","T"]  | "chr18_60371808_C_T"                      |
| chr18:60371817 | ["G","A"]  | "chr18_60371817_G_A"                      |
| chr18:60371827 | ["C","T"]  | "chr18_60371827_C_T"                      |
| chr18:60371842 | ["T","C"]  | "chr18_60371842_T_C"                      |
| chr18:60371854 | ["C","T"]  | "chr18_60371854_C_T"                      |
| chr18:60371970 | ["G","A"]  | "chr18_60371970_G_A"                      |
| chr18:60372015 | ["G","A"]  | "chr18_60372015_G_A;chr18_60372015_G_GT"  |
| chr18:60372043 | ["C","T"]  | "chr18_60372043_C_T"                      |
| chr18:60372111 | ["T","C"]  | "chr18_60372111_T_C"                      |
+----------------+------------+-------------------------------------------+

+----------+----------+---------------------+--------------+--------------+
|     qual | filters  | info.AF             | info.AQ      | info.AC      |
+----------+----------+---------------------+--------------+--------------+
|  float64 | set<str> | array<float64>      | array<int32> | array<int32> |
+----------+----------+---------------------+--------------+--------------+
| 4.10e+01 | NA       | [5.00e-06]          | [41]         | NA           |
| 4.50e+01 | NA       | [2.00e-06]          | [45]         | NA           |
| 4.60e+01 | NA       | [2.00e-06]          | [46]         | NA           |
| 5.30e+01 | NA       | [1.30e-02,3.50e-05] | [53,51]      | NA           |
| 5.50e+01 | NA       | [9.57e-04]          | [55]         | NA           |
| 4.70e+01 | NA       | [2.00e-06]          | [47]         | NA           |
| 5.00e+01 | NA       | [7.00e-06]          | [50]         | NA           |
| 4.80e+01 | NA       | [1.37e-04]          | [48]         | NA           |
| 5.40e+01 | NA       | [1.10e-04]          | [54]         | NA           |
| 4.50e+01 | NA       | [2.00e-06]          | [45]         | NA           |
| 4.80e+01 | NA       | [6.20e-05]          | [48]         | NA           |
| 5.60e+01 | NA       | [5.18e-04,7.00e-06] | [56,51]      | NA           |
| 5.30e+01 | NA       | [2.00e-02]          | [53]         | NA           |
| 4.70e+01 | NA       | [1.70e-05]          | [47]         | NA           |
+----------+----------+---------------------+--------------+--------------+

+---------+---------+-----------+-------------------+-------------------+
| info.AN | a_index | was_split | vep.assembly_name | vep.allele_string |
+---------+---------+-----------+-------------------+-------------------+
|   int32 |   int32 |      bool | str               | str               |
+---------+---------+-----------+-------------------+-------------------+
|      NA |       1 |     false | "GRCh38"          | "G/A"             |
|      NA |       1 |     false | "GRCh38"          | "T/C"             |
|      NA |       1 |     false | "GRCh38"          | "A/G"             |
|      NA |       1 |      true | "GRCh38"          | "T/G"             |
|      NA |       1 |     false | "GRCh38"          | "G/A"             |
|      NA |       1 |     false | "GRCh38"          | "C/T"             |
|      NA |       1 |     false | "GRCh38"          | "G/A"             |
|      NA |       1 |     false | "GRCh38"          | "C/T"           

In [87]:
not_found = ht.anti_join(found.key_by("protCons"))

In [88]:
not_found.show(-1)

2021-12-06 15:10:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:07 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:08 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:11 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:13 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:14 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:15 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:18 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:22 Hail: INFO: Ordering unsorted dataset with network shuffle


,
Variants,Category
str,int32
"""5'UTR 32bp vor A+G:A>C""",4
"""A135P""",2
"""D90N""",2
"""E61*""",2
"""P272L""",2
"""P48S""",4
"""S77L""",1
"""S77L""",1


In [89]:
no_label = mt.filter_rows(~hl.is_defined(mt.labels)).rows()
no_label = no_label.annotate(**no_label.vep.transcript_consequences[0])
no_label = no_label.drop("vep")
no_label.show(-1)

2021-12-06 15:10:25 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-12-06 15:10:32 Hail: INFO: Ordering unsorted dataset with network shuffle


+----------------+-------------------+
| locus          | alleles           |
+----------------+-------------------+
| locus<GRCh38>  | array<str>        |
+----------------+-------------------+
| chr18:60371355 | ["T","C"]         |
| chr18:60371356 | ["A","G"]         |
| chr18:60371358 | ["C","T"]         |
| chr18:60371359 | ["T","G"]         |
| chr18:60371365 | ["A","C"]         |
| chr18:60371374 | ["A","G"]         |
| chr18:60371377 | ["G","A"]         |
| chr18:60371378 | ["G","A"]         |
| chr18:60371382 | ["C","A"]         |
| chr18:60371386 | ["G","C"]         |
| chr18:60371387 | ["G","A"]         |
| chr18:60371389 | ["GA","G"]        |
| chr18:60371401 | ["T","C"]         |
| chr18:60371403 | ["A","C"]         |
| chr18:60371427 | ["T","A"]         |
| chr18:60371428 | ["C","T"]         |
| chr18:60371431 | ["G","A"]         |
| chr18:60371433 | ["C","T"]         |
| chr18:60371435 | ["C","G"]         |
| chr18:60371436 | ["C","T"]         |
| chr18:60371437 | ["G","A"]         |
| chr18:60371444 | ["A","G"]         |
| chr18:60371445 | ["T","A"]         |
| chr18:60371454 | ["G","T"]         |
| chr18:60371459 | ["G","A"]         |
| chr18:60371466 | ["G","A"]         |
| chr18:60371466 | ["G","T"]         |
| chr18:60371467 | ["A","G"]         |
| chr18:60371468 | ["A","G"]         |
| chr18:60371474 | ["CA","C"]        |
| chr18:60371474 | ["CATG","C"]      |
| chr18:60371477 | ["G","A"]         |
| chr18:60371499 | ["A","C"]         |
| chr18:60371500 | ["A","T"]         |
| chr18:60371507 | ["C","T"]         |
| chr18:60371513 | ["G","A"]         |
| chr18:60371513 | ["G","GCA"]       |
| chr18:60371520 | ["CAA","C"]       |
| chr18:60371522 | ["A","G"]         |
| chr18:60371532 | ["T","C"]         |
| chr18:60371538 | ["C","T"]         |
| chr18:60371541 | ["G","A"]         |
| chr18:60371555 | ["T","C"]         |
| chr18:60371577 | ["C","T"]         |
| chr18:60371587 | ["C","T"]         |
| chr18:60371588 | ["A","G"]         |
| chr18:60371593 | ["C","T"]         |
| chr18:60371594 | ["G","A"]         |
| chr18:60371596 | ["C","T"]         |
| chr18:60371598 | ["ATC","A"]       |
| chr18:60371601 | ["A","T"]         |
| chr18:60371603 | ["G","A"]         |
| chr18:60371615 | ["A","G"]         |
| chr18:60371618 | ["C","T"]         |
| chr18:60371619 | ["G","A"]         |
| chr18:60371619 | ["G","T"]         |
| chr18:60371624 | ["C","T"]         |
| chr18:60371628 | ["A","G"]         |
| chr18:60371631 | ["T","C"]         |
| chr18:60371632 | ["T","C"]         |
| chr18:60371635 | ["C","T"]         |
| chr18:60371637 | ["C","T"]         |
| chr18:60371638 | ["C","T"]         |
| chr18:60371643 | ["C","T"]         |
| chr18:60371644 | ["G","A"]         |
| chr18:60371644 | ["G","C"]         |
| chr18:60371659 | ["C","T"]         |
| chr18:60371660 | ["G","A"]         |
| chr18:60371671 | ["C","T"]         |
| chr18:60371673 | ["A","G"]         |
| chr18:60371684 | ["G","A"]         |
| chr18:60371695 | ["C","A"]         |
| chr18:60371697 | ["A","G"]         |
| chr18:60371701 | ["G","A"]         |
| chr18:60371705 | ["C","T"]         |
| chr18:60371707 | ["T","C"]         |
| chr18:60371714 | ["A","G"]         |
| chr18:60371720 | ["A","G"]         |
| chr18:60371731 | ["G","C"]         |
| chr18:60371742 | ["G","A"]         |
| chr18:60371744 | ["G","T"]         |
| chr18:60371751 | ["A","G"]         |
| chr18:60371752 | ["T","C"]         |
| chr18:60371763 | ["C","T"]         |
| chr18:60371767 | ["T","C"]         |
| chr18:60371771 | ["G","A"]         |
| chr18:60371774 | ["A","T"]         |
| chr18:60371780 | ["A","G"]         |
| chr18:60371783 | ["A","G"]         |
| chr18:60371790 | ["TA","T"]        |
| chr18:60371798 | ["G","A"]         |
| chr18:60371800 | ["A","T"]         |
| chr18:60371801 | ["C","A"]         |
| chr18:60371812 | ["A","G"]         |
| chr18:60371816 | ["C","A"]         |
| chr18:60371816 | ["C","T"]         |
| chr18:60371830 | ["A","G"]         |
| chr18:60371832 | ["A